In [1]:
import os
import shutil
import random
from sklearn.model_selection import train_test_split
from collections import defaultdict


# Data loading

In [3]:
# Set random seed for reproducibility
random.seed(42)

# Define paths
base_dir = "."
content_base_dir = os.path.join(base_dir, "Content", "images", "images")
style_dir = os.path.join(base_dir, "style")

# Output directories for splits
content_train_dir = os.path.join(base_dir, "Content", "train")
content_val_dir = os.path.join(base_dir, "Content", "validation")
content_test_dir = os.path.join(base_dir, "Content", "test")
style_train_dir = os.path.join(base_dir, "style", "train")
style_val_dir = os.path.join(base_dir, "style", "validation")
style_test_dir = os.path.join(base_dir, "style", "test")

# Create output directories if they don't exist
for directory in [content_train_dir, content_val_dir, content_test_dir, 
                  style_train_dir, style_val_dir, style_test_dir]:
    os.makedirs(directory, exist_ok=True)

# Helper function to get image paths from subfolders
def get_image_paths(base_directory):
    image_paths = defaultdict(list)
    for category in os.listdir(base_directory):
        category_path = os.path.join(base_directory, category)
        if os.path.isdir(category_path):
            image_paths[category] = [os.path.join(category_path, f) for f in os.listdir(category_path)
                                    if f.lower().endswith((".jpg", ".jpeg", ".png"))]
    return image_paths

# Load all image paths
content_images = get_image_paths(content_base_dir)
style_images = [os.path.join(style_dir, f) for f in os.listdir(style_dir) 
                if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# Print initial counts
total_content = sum(len(images) for images in content_images.values())
print(f"Total content images: {total_content}")
print(f"Total style images: {len(style_images)}")

Total content images: 35093
Total style images: 5000


# Sampling

In [4]:
target_content = 7000
target_style = 3000

# Sample content proportionally from each category
content_sampled = []
total_images = sum(len(images) for images in content_images.values())
for category, images in content_images.items():
    if total_images > 0:
        category_target = int((len(images) / total_images) * target_content)
        category_target = min(category_target, len(images))  # Don't exceed available images
        sampled = random.sample(images, category_target) if len(images) >= category_target else images
        content_sampled.extend(sampled)

# Adjust if still over 7,000 due to rounding
if len(content_sampled) > target_content:
    content_sampled = random.sample(content_sampled, target_content)
elif len(content_sampled) < target_content and len(content_images) > len(content_sampled):
    additional = random.sample([img for cat in content_images.values() for img in cat 
                              if img not in content_sampled], target_content - len(content_sampled))
    content_sampled.extend(additional)

# Sample style images
style_sampled = random.sample(style_images, target_style) if len(style_images) >= target_style else style_images

print(f"Sampled content images: {len(content_sampled)}")
print(f"Sampled style images: {len(style_sampled)}")


Sampled content images: 6998
Sampled style images: 3000


# Splitting

In [ ]:
# Content split
content_train, content_temp = train_test_split(content_sampled, train_size=0.7, random_state=42)
content_val, content_test = train_test_split(content_temp, test_size=0.5, random_state=42)

# Style split
style_train, style_temp = train_test_split(style_sampled, train_size=0.7, random_state=42)
style_val, style_test = train_test_split(style_temp, test_size=0.5, random_state=42)

print(f"Content split - Train: {len(content_train)}, Val: {len(content_val)}, Test: {len(content_test)}")
print(f"Style split - Train: {len(style_train)}, Val: {len(style_val)}, Test: {len(style_test)}")

# Step 3: Move images to respective split directories
def move_images(image_paths, destination_dir):
    for img_path in image_paths:
        try:
            shutil.move(img_path, os.path.join(destination_dir, os.path.basename(img_path)))
        except Exception as e:
            print(f"Failed to move {img_path}: {e}")

# Move content images
move_images(content_train, content_train_dir)
move_images(content_val, content_val_dir)
move_images(content_test, content_test_dir)

# Move style images
move_images(style_train, style_train_dir)
move_images(style_val, style_val_dir)
move_images(style_test, style_test_dir)

# Step 4: Verify splits before deletion
print("Verifying splits before deleting remaining images...")
print(f"Content train images: {len(os.listdir(content_train_dir))}")
print(f"Content validation images: {len(os.listdir(content_val_dir))}")
print(f"Content test images: {len(os.listdir(content_test_dir))}")
print(f"Style train images: {len(os.listdir(style_train_dir))}")
print(f"Style validation images: {len(os.listdir(style_val_dir))}")
print(f"Style test images: {len(os.listdir(style_test_dir))}")

Content split - Train: 4898, Val: 1050, Test: 1050
Style split - Train: 2100, Val: 450, Test: 450


KeyboardInterrupt: 

In [ ]:
# Reload remaining content images
remaining_content_images = get_image_paths(content_base_dir)
remaining_style_images = [os.path.join(style_dir, f) for f in os.listdir(style_dir) 
                         if f.lower().endswith((".jpg", ".jpeg", ".png"))]

# Delete remaining content images
print("Deleting remaining content images...")
for category, images in remaining_content_images.items():
    category_path = os.path.join(content_base_dir, category)
    for img_path in images:
        try:
            os.remove(img_path)
            print(f"Deleted {img_path}")
        except Exception as e:
            print(f"Failed to delete {img_path}: {e}")
    # Remove category folder if empty
    if not os.listdir(category_path):
        shutil.rmtree(category_path)
        print(f"Removed empty category folder: {category_path}")

# Delete remaining style images
print("Deleting remaining style images...")
for img_path in remaining_style_images:
    try:
        os.remove(img_path)
        print(f"Deleted {img_path}")
    except Exception as e:
        print(f"Failed to delete {img_path}: {e}")

# Final verification
print("Final folder structure after deletion:")
print(f"Content train images: {len(os.listdir(content_train_dir))}")
print(f"Content validation images: {len(os.listdir(content_val_dir))}")
print(f"Content test images: {len(os.listdir(content_test_dir))}")
print(f"Style train images: {len(os.listdir(style_train_dir))}")
print(f"Style validation images: {len(os.listdir(style_val_dir))}")
print(f"Style test images: {len(os.listdir(style_test_dir))}")

NameError: name 'content_train' is not defined